# Using py2neo to work with neo4j

Py2neo is a client library/toolkit for working with Neo4j with Python. 
This notebook exemplificates several aspects of the library. 

The complete documentation can be found here: http://py2neo.org/v3/

### Connecting to a neo4j database instance

More details at: http://py2neo.org/v3/database.html#the-graph

In [1]:
from py2neo import Graph

### Using default parameters -- localhost:7474 no login/passwd required
#graph = Graph()

### Using url only (the notation "http://<host>:<port>" is mandatory)
#graph = Graph("http://localhost:7474/db/data/")
#graph = Graph("http://neo4j.lis.ic.unicamp.br:80/db/data/")

### http 
#graph = Graph(host ="localhost", http_port = 7474)
#graph = Graph("http://neo4j.lis.ic.unicamp.br:80/db/data/")

### https
#graph = Graph(host ="localhost", http_port = 7473)
#graph = Graph("http://neo4j.lis.ic.unicamp.br:443/db/data/")

### authentication 
#graph = Graph(host = "localhost", http_port = 7474, user="neo4j", password="123")

### For neo4j's new Bolt binary protocol (available in Neo4j 3.0+) see the documentation link 

### My parameters
#graph = Graph("http://neo4j.lis.ic.unicamp.br:80/db/data/", )
graph = Graph(host = "neo4j.lis.ic.unicamp.br", http_port = 80, bolt = False,  secure = False)


if(graph):
    print ("Connected")
    print (graph)

graph.begin(autocommit=True)
graph.delete_all()

Connected
<Graph uri='http://neo4j.lis.ic.unicamp.br:80/db/data/'>


### Creating Nodes

In [2]:
from py2neo import Node

### Examples of nodes labeled as Person
john = Node("Person", name="John", surname="Doe", age=24)
jane = Node("Person", name="Jane", surname="Doe", age=28)

### Examples of nodes labeled as Movie
memento = Node("Movie", title="Memento", year=2001)
ib      = Node("Movie", title="Inglourious Basterds", year=2009)
titanic = Node("Movie", title="Titanic", year=1997)

### Examples of nodes labeled as Serie
twd = Node("Serie", title="The Walking Dead", year=2010)

### Creatin/persisting the nodes
graph.create(john | jane | memento | ib | titanic | twd)

Using Pandas to retrieve the created nodes

In [10]:
from pandas import DataFrame

DataFrame(graph.data("MATCH n WHERE n:Person OR n:Movie OR n:Serie RETURN n, labels(n) as label order by label[0]"))

TypeError: data argument can't be an iterator

Using [Nicole's](https://github.com/nicolewhite) python [script](https://github.com/nicolewhite/neo4j-jupyter/blob/master/scripts/vis.py) for rendering the graph using [Viz](http://visjs.org/).


In [4]:
from resources.scripts.vis import draw

options = {"Person": "name", "Movie": "title", "Serie": "title"}
draw(graph, options,  physics=True)

### Creating Relationships

In [5]:
from py2neo import Relationship

### Examples of relationships 
graph.create(Relationship(john, "IS_MARRIED_WITH", jane))

graph.create(Relationship(john, "LIKES", memento))
graph.create(Relationship(john, "LIKES", twd))
graph.create(Relationship(john, "LIKES", ib))

graph.create(Relationship(jane, "LIKES", titanic))
graph.create(Relationship(jane, "LIKES", twd))



In [6]:
from resources.scripts.vis import draw

draw(graph, options,  physics=False)

### Cypher

In [11]:
from pandas import DataFrame


query = """
MATCH (person:Person)-[:LIKES]->(movie:Movie)
RETURN person.name AS name, movie.title AS movie
"""

data = graph.run(query)

for d in data:
    print(d)



('name': 'Jane', 'movie': 'Titanic')
('name': 'John', 'movie': 'Inglourious Basterds')
('name': 'John', 'movie': 'Memento')


### Parameterized Cypher

In [15]:
query = """
MATCH (person:Person)-[:LIKES]->(movie:Movie)
WHERE person.surname = {surname}
RETURN person.name AS name, AVG(movie.year) AS avg_calories
"""

data = graph.run(query, surname="John")

for d in data:
    print(d)